In [4]:
from src.preprocessing.text_preprocessing import clean_text, transform_countvec, transform_tfidf
import src.models.models as models
import numpy as np
import pandas as pd

In [5]:
train_raw = pd.read_csv("data/train.csv")
test_raw = pd.read_csv("data/test.csv")
y_test = pd.read_csv("data/submit.csv")

train_clean = clean_text(train_raw)
test_clean = clean_text(test_raw)
y_train = train_clean["label"]

X_train, X_test, words = transform_tfidf(train_clean["text"], test_clean["text"])

X_train = X_train.todense()

c:\Users\aupab\Desktop\mathml_final_proj\mathml_finalproj\src\preprocessing\text_preprocessing.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  col = col.str.replace('[^\w\s]', '')\


In [6]:
y_train = y_train.to_numpy()

In [7]:
X_train = X_train[:500,]
y_train = y_train[:500]
n, p = X_train.shape
indices = np.random.permutation(n)
X_train = X_train[indices,:]
y_train = y_train[indices]
folds = 5
n_alphas = 10
lambdas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]

avg_errors_by_penalty = np.empty((n_alphas + 1,len(lambdas)))

In [27]:
import numpy as np


def gradient_descent(x_train, x_dev, y_train, y_dev, n_alphas, lambda_):
    '''
    '''

    alphas = np.linspace(0, 1, n_alphas+1)
    errors = []

    for alpha in alphas:
        w = sgd(x_train, y_train, alpha, lambda_, 1e-4)
        pred = (1 / (1 + np.exp(-x_dev.dot(w)))).A1
        

        errors = (-y_dev*np.log(pred)) - ((1-y_dev)*np.log(pred))

        error_alpha_i  = np.mean(errors)

    return error_alpha_i


def sgd(X, y, alpha, lambda_, eta): 
    '''
    '''
    n, p = X.shape
    l1 = alpha*lambda_
    l2 = (1-alpha)*lambda_

    w = np.zeros(p)
    dw = np.zeros(p)

    for _ in range(100):
        i = np.random.randint(0, n)
        x_i = X[i]
        y_i = y[i]

        pred_i = 1 / (1 + np.exp(-x_i.dot(w)))

        dw = -1* (y_i - pred_i)* x_i + l1*np.sign(w) + 2*l2*w

        w = w - eta * dw
        w = w.A1

    return w






In [28]:
for i, lambda_ in enumerate(lambdas):
    errors = np.empty((n_alphas+ 1,folds))
    buckets = list(range(0, n + 1, n//folds))
    
    for k in range(folds):
        val1 = buckets[k]
        val2 = buckets[k+1]

        x_dev = X_train[val1:val2,:]
        x_train = np.delete(X_train, np.s_[val1:val2], axis=0)

        y_dev = y_train[val1:val2]
        y_train2 = np.delete(y_train, np.s_[val1:val2], axis=0)

        errors[:,k] = gradient_descent(x_train, x_dev, y_train2, y_dev,n_alphas, lambda_)

    avg_errors_by_penalty[:,i] = np.mean(errors, axis=1)

In [29]:
print(avg_errors_by_penalty)

[[0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]
 [0.69308609 0.69314638 0.69315181 0.69316351 0.69316381 0.69318051
  0.69317203]]
